# dataset/ingest

In [15]:
import os
from tqdm import *
from abcli import file
from abcli.modules import objects
from abcli import fullname
from abcli.plugins import cache
from roofAI.semseg.utils import visualize
from roofAI.dataset import RoofAIDataset, DatasetKind, MatrixKind
from roofAI.dataset.ingest.AIRS import ingest_AIRS
from IPython.display import Image
from abcli import string
from abcli.plugins import seed
import numpy as np

import abcli.logging
import logging

logger = logging.getLogger()

logger.info(f"{fullname()}, built on {string.pretty_date()}")

abcli-7.2382.1-2023-06-aws-batch-a, built on 20 October 2023, 22:48:45


In [2]:
cache_object_name = cache.read("roofAI_ingest_AIRS_cache")
ingest_object_name=objects.unique_object("roofAI-AIRS-ingest")

using cache: roofAI_ingest_AIRS_cache_2023-10-15-15-53-41-09195


In [4]:
cache_object_path = objects.object_path(cache_object_name)
ingest_object_path = objects.object_path(ingest_object_name)

```python
assert ingest_AIRS(
    cache_path=cache_object_path,
    ingest_path=ingest_object_path,
    {
        "test": 10,
        "train": 10,
        "val": 10
    }
)
```

In [9]:
counts = {
    "test": 10,
    "train": 10,
    "val": 10
}

chip_width = 480
chip_height = 360
chip_overlap = 0.1

In [5]:
dataset = RoofAIDataset(cache_object_path)

RoofAIDataset[DatasetKind.AIRS:AIRS](roofAI_ingest_AIRS_cache_2023-10-15-15-53-41-09195): 3 subset(s): 95 X test + 857 X train + 94 X val


In [16]:
for subset in tqdm(counts.keys()):
    count = 0
    for record_id in dataset.subsets[subset]:
        for matrix_kind in list(MatrixKind):
            input_matrix = dataset.get_matrix(
                subset,
                record_id,
                matrix_kind
            )

            sliced_matrix = slice(
                input_matrix,
                chip_width,
                chip_height,
                chip_overlap,
                path=...
            )
        
        count += len(sliced_matrix)
        if count >= counts[subset]:
            break


100%|██████████| 3/3 [00:00<00:00, 3374.34it/s]

MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind.IMAGE
MatrixKind.MASK
MatrixKind

In [6]:
seed.log(ingest_object_path)

abcli select /Users/kamangir/storage/abcli/roofAI-AIRS-ingest-2023-10-20-22-42-17-60779; open .


In [7]:
# END